In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format
# pd.options.display.int_format ='{:,}'.format

import os
from operator import itemgetter    
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')


In [3]:
import_csv_file = '../_data/operations_fulldata.csv'
df = pd.read_csv(import_csv_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200593 entries, 0 to 200592
Data columns (total 55 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   op_id              200593 non-null  int64  
 1   subject_id         200593 non-null  int64  
 2   hadm_id            200593 non-null  int64  
 3   opdate             200593 non-null  int64  
 4   age                200593 non-null  int64  
 5   sex                200593 non-null  object 
 6   weight             198158 non-null  float64
 7   height             199124 non-null  float64
 8   race               200593 non-null  object 
 9   asa                194133 non-null  float64
 10  emop               200593 non-null  int64  
 11  department         200593 non-null  object 
 12  antype             200593 non-null  object 
 13  icd10_pcs          200593 non-null  object 
 14  category_desc      200593 non-null  object 
 15  desc_short         200593 non-null  object 
 16  ca

In [7]:
# calculate teh operation durations and add to DF
df['op_duration']=df['orout_time']-df['orin_time']
df['LOS']=(df['discharge_time']-df['orout_time'])/1440 # LOS is time following surgery to dishcarge converted to days. 
operations_info = df.loc[:,['op_id','category_id','category_desc','desc_short', 'icd10_pcs', 'orin_time','orout_time', 'op_duration', 'LOS']]

operations_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200593 entries, 0 to 200592
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   op_id          200593 non-null  int64  
 1   category_id    200593 non-null  object 
 2   category_desc  200593 non-null  object 
 3   desc_short     200593 non-null  object 
 4   icd10_pcs      200593 non-null  object 
 5   orin_time      200593 non-null  int64  
 6   orout_time     200593 non-null  int64  
 7   op_duration    200593 non-null  int64  
 8   LOS            200593 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 13.8+ MB


In [25]:
import pandas as pd

# Group the DataFrame by 'icd10_pcs' and 'desc_schort' and calculate descriptive statistics for 'LOS'
descriptive_stats = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].describe()

# use the agg() method with a custom list of percentiles
percentiles = [0.1, 0.9]  # 
percentile_stats = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].quantile(percentiles).unstack()

# # Concatenate the percentile stats to the descriptive stats
descriptive_stats = pd.concat([descriptive_stats, percentile_stats], axis=1)

# # Rename the percentile columns for clarity
descriptive_stats = descriptive_stats.rename(columns={0.1: '10%', 0.9: '90%'})

# MODE or additional quantiles if needed
# descriptive_stats['mode'] = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].agg(pd.Series.mode)

descriptive_stats.sort_values(by='std', ascending=False).head(20)
# descriptive_stats.to_csv('los.csv')


,,count,mean,std,min,25%,50%,75%,max,10%,90%
icd10_pcs,desc_short,,,,,,,,,,
0B110,Bypass Trachea to Esophagus with Intralum Dev,769,95,289,0,13,25,64,"3,711",6,170
00900,Drainage of Brain with Drainage Device,651,62,270,1,8,21,39,"3,728",2,88
00160,Bypass Cereb Vent to Nasophar with Autol Sub,"1,719",99,206,1,8,25,93,"1,778",5,259
00P60,Removal of Other Device from Cereb Vent,46,123,188,8,13,27,58,475,8,475
001U0,Bypass Spinal Canal to Urinary Tract w Nonaut Sub,87,58,174,3,13,22,33,"1,157",5,67
02QA0,Repair Heart,135,28,134,1,7,10,17,"1,553",4,38
0BQ10,Repair Trachea,112,22,119,0,1,3,8,896,1,20
02VP0,Restriction of Pulm Trunk with Intralum Dev,528,27,113,2,5,6,8,825,4,24
00Q60,Repair Cerebral Ventricle,36,55,106,2,4,17,29,314,3,314
